**NOTE: Before we get started with the exercises, it is useful to get acquainted with some core concepts of reinforcement learning.
If you are already an expert or you want to get started with the exercises right away, you can skim or skip this read for now and refer back to it later.**

# An Introduction to Deep Reinforcement Learning

> By Jonas Busk ([jbusk@dtu.dk](mailto:jbusk@dtu.dk))

Anyone with an interest in machine learning and artificial intelligence is unlikely to have missed, that the field of reinforcement learning (RL) has achieved impressive results in recent years, such as reaching human level performance in Atari 2600 games [2] and defeating a world champion in the complex board game Go [3].
Some of this success can be attributed to Moore's law of increase in computer power and the overall progress and success of deep learning, which have given rise to new powerful RL algorithms.
Many of these algorithms, however, are built on ideas from classic RL.
Therefore, it is useful to review some of this background in order to better understand what is happening in deep RL. 

This notebook provides a brief introduction to deep RL by first reviewing some core ideas forming the basis of RL and then presenting some general approaches to solving RL problems.
The reader is assumed to already be familiar with artificial neural networks and backpropagation, along with the relevant math, though not necessarily know how to apply these concepts in the context of RL.

## What is reinforcement learning?

RL is learning by interacting with an environment so as to maximize some reward [1]. This is different from other types of machine learning, where most often you learn from a fixed set of training examples in a supervised or unsupervised fashion.
Instead, an RL-agent gathers experience by choosing actions and learning from the outcomes of its decisions in an ever changing dataset. Essentially, RL is learning by trial and error.

An example could be an agent learning to play a game by repeatedly interacting with the game and observing its score.
The learning task is then for the agent to increase the probability of selecting "good" actions and decrease the probability of "bad" actions in order to master the game and maximize its overall expected amount of reward.
Learning a good strategy involves exploring the outcome of selecting different actions and to some extent exploit the knowledge it has already learned in order to progress in the game.

## The reinforcement learning problem

So how do we formalize a RL problem?
Consider an **agent** navigating an **environment** [1]. 
The environment is in a certain **state**, $s \in \mathcal{S}$, and the agent can perform **actions**, $a \in \mathcal{A}(s)$, which transform the environment through a **transition distribution**, $\mathcal{T}(s'|s,a)$, resulting in a new state, $s'$, and a **reward** denoted by $r$ or $\mathcal{R}(s)$ or sometimes $\mathcal{R}(s,a,s')$, for performing that action.
In this new state, the agent can perform another action, and so on.
This system is illustrated in the figure below. 

![reinforcement-learning](images/reinforcement-learning.png)

The set of states, set of actions and rules for transitioning between states and assigning rewards along with a distribution of starting states, $p(s)$, make up a **Markov decision process (MDP)**.
One transition of the MDP can be described by a state, the action performed from that state along with the resulting reward and next state, denoted $(s,a,r,s')$.
In this framework, it is an important assumption that the probability of the next state, $s'$, depends only on the current state and action, $(s,a)$, and not of the sequence of states and actions that preceded them. This is also known as the **Markov property**.

Although many RL algorithms build on this assumption, it requires the states to be fully observable, which is not always true. In a partially observable MDP (POMDP) the agent receives an **observation** $o \in \Omega$, representing the agent's perception of the state, where the distribution of the observation, $p(o'|s',a)$, depends on the current state and previous action. You will often encounter the terms *observation* and *state* used interchangeably.

## The credit assignment problem

When an agent interacts with its environment and receives rewards repeatedly, it can be difficult to know exactly what actions helped achieving a particular reward. In many cases the reward is not a direct result of the most recent action, but instead is caused by earlier decisions, i.e. the reward is delayed in time. This is known as the **credit assignment problem** [1]. 
To address this issue, we need to take into account not only the immediate rewards, but also the future rewards, when evaluating the utility of an action. 

Given a series of transitions consisting of $T$ timesteps, it is straight forward to compute the **total future reward** or **return** from time, $t$:

$$
%R_t = r_t + r_{t+1} + r_{t+2} + \dots + r_T \ .
R_t = r_t + r_{t+1} + r_{t+2} + \dots + r_T = \sum_{k=0}^{T-t} r_{t+k} \ .
$$

In a stochastic environment we cannot be absolutely sure of rewards happening far into the future. Even if we repeat the same series of actions, it can lead to a different result. In fact, the uncertainty in the rewards grows the farther into the future we look. Therefore it is common to use **discounted future reward** also known as **discounted return** instead, where rewards are weighted by uncertainty:

$$
%R_t = r_t + \gamma r_{t+1} + \gamma^2 r_{t+2} + \dots + \gamma^{T} r_T = r_t + \gamma R_{t+1} \ ,
R_t = r_t + \gamma r_{t+1} + \gamma^2 r_{t+2} + \dots + \gamma^{T} r_T = \sum_{k=0}^{T-t} \gamma^k r_{t+k} \ ,
$$

where $\gamma \in [0,1]$ is the **discount rate**. If $\gamma = 1$, there is no discount, which is suitable for deterministic environments where repeating a series of actions is certain to lead to the same rewards. 
If we set $\gamma = 0$, only immediate rewards are considered, resulting in a very short-sighted, greedy strategy. Usually something like $\gamma = 0.9$ works well.

When the MDP can naturally be considered to consist of episodes, where the state is reset after $T$ timesteps, then it is said to be **episodic** and the sequence of states, actions and rewards in each episode make up a **trajectory** or **rollout**.
It is also possible to have $T = \infty$ in situations where there is no natural notion of a final step. Then the task is said to be continuing and choosing $\gamma < 1$ prevents an infinite sum of rewards in this case.

## Policy

The rule for how an agent selects its actions is called a **policy**, typically denoted by $\pi$, and generally is a mapping from states to actions [1]: 

$$
\pi: \mathcal{S} \to \mathcal{A} \ .
$$

Policies can be either **deterministic**, when a certain state always maps to a certain action, or **stochastic**, when a state maps to a probability distribution over available actions, from which an action can be sampled:

$$
\pi: \mathcal{S} \to p(\mathcal{A}=a|\mathcal{S}=s) \ .
$$

The ultimate goal of RL is to find the optimal policy, $\pi^*$, that maximizes the expected return from all states. 
There is always at least one optimal policy that is better than or equal to all other policies in terms of expected return from all states:

$$
\pi^* = \operatorname{argmax}_\pi \mathbb{E}[R|\pi] \ .
$$

In practice it is often difficult to find the optimal policy (or even know if you have found it). However, in many cases a good estimate can be an acceptable solution.

## Value functions

Value functions estimate the value (expected return) and thereby the goodness of being in a given state [1].
Of course the rewards an agent can expect to see in the future depend on its actions. Therefore value functions are defined with respect to a particular policy.
There are a few value functions important to reinforcement learning that we will go over here.

The **state-value function**, $V^{\pi}(s)$, is the expected return from state $s$ when following policy $\pi$:

$$
V^{\pi}(s) = \mathbb{E}_{\pi}[R|s] \ .
$$

If a state-value function is available, the best policy given $V^{\pi}$, $\pi'$, can be retrieved by greedily choosing the action, $a$, that maximizes the expected value of the next state, $s'$: 

$$
\pi'(s|V^{\pi}) = \argmax_{a}\mathbb{E}_{s' \sim \mathcal{T}(s'|s,a)}[V^{\pi}(s')] \ .
$$

Interestingly, $\pi'$ is guaranteed to be *at least as good* as $\pi$.

Since the exact transition distribution, $\mathcal{T}$, is unavailable in most RL settings, it can be more useful to consider the **action-value function** or **quality function**, $Q^{\pi}(s,a)$, which is the expected return from state $s$ when selecting action $a$ and then following policy $\pi$:

$$
Q^{\pi}(s,a) = \mathbb{E}_{\pi}[R|s,a] \ ,
$$

The best policy given $Q^{\pi}$, $\pi'$, can be retrieved by greedily choosing the action, $a$, that maximizes $Q^{\pi}(s,a)$:

$$
\pi'(s|Q^{\pi}) = \argmax_{a}\mathbb{E}[Q(s,a)] \ .
$$

As with $V^{\pi}$ above, $\pi'$ is guaranteed to be *at least as good* as $\pi$.
Notice how this new policy no longer depends on the transition distribution, $\mathcal{T}$, and can therefore be retrieved without knowing the dynamics of the environment. 

We can also retrieve the corresponding $V^{\pi}(s)$ by maximizing $Q^{\pi}(s,a)$ over actions:

$$
V^{\pi}(s) = \max_{a} Q^{\pi}(s,a) \ .
$$

Another value function of interest is the **advantage function**, $A^{\pi}(s,a)$.
Instead of producing the absolute action-value like $Q^{\pi}$, the advantage function provides the relative action-value:

$$
A^{\pi}(s,a) = Q^{\pi}(s,a) - V^{\pi}(s) \ ,
$$

representing the relative value of choosing one action over another.
As with $Q^{\pi}$, the best policy given $A^{\pi}$, $\pi'$, can be retrieved by selecting the action, $a$, that maximizes $A^{\pi}(s,a)$:

$$
\pi'(s|A^{\pi}) = \argmax_{a}\mathbb{E}[A^{\pi}(s,a)] \ .
$$

Also here $\pi'$ is guaranteed to be *at least as good* as $\pi$.

As mentioned in the previous section, the goal of RL is to find an optimal policy, $\pi^*$.
There can be more than one optimal policy, but all optimal policies share the same optimal value functions:

$$
\begin{align}
V^*(s) &= \max_{\pi} V^{\pi}(s) \\
Q^*(s,a) &= \max_{\pi} Q^{\pi}(s,a) \ ,
\end{align}
$$

for all $s \in \mathcal{S}$ and $a \in \mathcal{A}(s)$.

Generally, value functions can be estimated from experience by evaluating the corresponding policies in their respective environment and adjusting the value function to better match the observed rewards. 

As we will see, the value functions presented here play an important role in many RL solutions.

## Exploration-exploitation dilemma

RL agents need to explore their environment in order to assess its reward structure. After some exploration, the agent might have learned a rewarding set of actions, but without further exploration it cannot know if there exists an even better strategy. So when should the agent stop exploring and start exploiting what it has learned so far, in order to maximize its total reward? This is known as the **exploration-exploitation dilemma**. Different RL strategies approach this differently, but most solutions depend on heuristics, generally starting with exploration and gradually turning to more exploitation. 
The well known multi-armed bandit problem shows an example of this dilemma. 
This section describes some common solutions within RL.

The simplest action selection rule is to select the action with the highest estimated value. This method always exploits current knowledge greedily to maximize immediate reward and never explores alternative actions to see if they in fact turn out better [1]. 
A simple way of introducing exploration is to act greedily most of the time, but with small probability, $\epsilon$, select an action uniformly at random. This is known as an **epsilon-greedy** strategy, which despite its simplicity is applied in many modern RL algorithms. 
In practice, $\epsilon$ often starts high and is then gradually decreased to reduce exploration and turn to more exploitation as learning progresses.

In cases where the policy provides a probability distribution over actions (stochastic policy), one can instead select an action by sampling from the distribution. This is sometimes refereed to as **softmax action selection**. 
The greedy action is still selected with highest probability, but unlike the epsilon-greedy approach, the second best action is sampled with higher probability than the estimated worst action, which can be important in cases where some actions are far better than others.
With this approach, a policy can be written as:

$$
\pi(a=j|s) = \frac{e^{p(a=j|s)/\tau}}{\sum_{i \in \mathcal{A}(s)} e^{p(a=i|s)/\tau}} \ ,
$$

where $\tau$ is called the temperature and a small $\tau$ results in a more greedy policy while a large $\tau$ makes actions more equiprobable.
A similar effect can be achieved by instead adding some random number, $\epsilon_j$, (scaled by $\tau$) to each $p(a=j|s)$ and selecting the action with the highest resulting value:

$$
\pi(s) = \argmax_j p(a=j|s) + \epsilon_j \ .
% where $\epsilon_j \sim N(0,\tau)$.
$$ 


An inherent property of the sampling approach is that it will gradually reduce exploration and start exploiting as it learns and becomes more confident in the most rewarding actions. 
It is not clear which of the approaches presented here is better and it might depend on the specific learning task. 

## Reinforcement learning methods
    
There are two main approaches to solving RL problems: **value function-based** methods, based on estimating value functions, and **policy search** methods, based on directly searching for the best policy [4]. Then there are also hybrid approaches employing both value functions and policy search. 
Finally, there are a wide range of extensions which can be applied across these basic approaches.

### Value function-based methods

Value function-based methods use an estimated value function to derive a policy. 
They generally start with a random value function and iteratively improve it to learn a solution. 

For example Q-learning learns a policy by estimating the $Q$-function.
To learn $Q$ we exploit the Markov property and define the Q-function as a Bellman equation known from dynamic programming:

$$
Q(s,a) = \mathbb{E}_{s'}[r + \gamma \, Q(s',\pi(s'))] \ .
$$

This means that $Q$ can be improved by using the current estimate of $Q$ to improve the estimate of $Q$, also known as bootstrapping:

$$
Q(s,a) \leftarrow Q(s,a) + \alpha \delta \ ,
$$

where $\alpha$ is the learning rate and $\delta = Y - Q(s,a)$ is the temporal difference (TD) error. 
Here $Y$ is the target value, in Q-learning defined as $r + \gamma \max_{a'} Q(s',a')$.
The target value can be defined in different ways giving rise to different algorithms, but is 
generally estimated from interactions with the environment.
This update rule can be applied iteratively to improve the estimate of $Q$ by minimizing the TD error.

Other value function-based methods rely on learning the advantage function, $A$, instead of $Q$, since intuitively it is easier to learn the relative action-values than the absolute values.
Applying the advantage is similar to removing a baseline or average from the action-values and also helps to reduce variance.

An alternative to bootstrapping is to create a Monte Carlo estimate of the expected return from a state, $V(s)$, by averaging the return from multiple rollouts of the current policy.
This strategy does not rely on the Markov property, but can only be applied in episodic environments.
Methods exist that combine Monte Carlo evaluation and bootstrapping.


### Policy search methods

Policy methods do not rely on value-functions, but instead try to learn a policy directly.
They typically employ a parameterized policy, $\pi_\theta$, whose parameters, $\theta$, are updated iteratively to maximize the expected return $\mathbb{E}[R|\theta]$.

Policy updates can either be gradient-based, taking the gradient of the expected return with regards to the parameters (policy gradient), or gradient-free, which typically relies on evolution-based strategies.
Previously, gradient-free optimization was believed to only effectively solve low dimensional problems while gradient-based methods had been successfully applied to larger problems, but recently evolutionary strategies have been shown to solve complex problems as well [5].  

In the gradient-based approach, the gradient cannot be evaluated analytically, but the expected return can be estimated through Monte Carlo sampling by averaging over rollouts produced with the current policy.
This estimator of the gradient is commonly known as the REINFORCE rule.
In its general form, the REINFORCE rule can be used to compute the gradient of an expectation over a function $f$ of a random variable $X$ with respect to parameters $\theta$:

$$
\nabla_\theta \mathbb{E}_X[f(X;\theta)] = \mathbb{E}_X[f(X;\theta)\nabla_\theta \log p_\theta(X)] \ ,
$$

where in the RL case, $f$ is the score function and $p_\theta$ is the policy.


When constructing a policy directly, it is common to output a probability distribution over available actions given the current state.
This allows for a stochastic policy from which actions can be sampled.


### Hybrid approaches

It is possible to combine value functions with an explicit representation of the policy, resulting in what is known as **actor-critic methods**, where the actor is the policy and critic is a value function.
In this setup, the actor learns from feedback provided by the critic in form of the TD error.
The difference from the regular policy gradient approach presented above is that actor-critic methods utilize a learned value function instead of a Monte Carlo estimate.

![actor-critic](images/actor-critic.png)

Because hybrid methods combine value function and policy search methods they can benefit from improvements in both areas.

### Deep reinforcement learning

For a simple problem with a finite state space, we could easily represent a value function as a table listing the value of each action for every state. 
But if for example the state consists of real numbers there exist an infinite number of states and the table approach is no longer feasible.
In general, deep RL is based on training artificial neural networks to approximate either a value function, the policy or a combination of both.

## References

1. Sutton and Barto, *Reinforcement Learning: An Introduction*, 1998.
2. Mnih et al., *Human-Level Control through Deep Reinforcement Learning*, 2015
3. Silver et al., *Mastering the Game of Go with Deep Neural Networks and Tree Search*, 2016.
4. Arulkumaran et al., *A Brief Survey of Deep Reinforcement Learning*, 2017.
5. Salimans et al., *Evolution Strategies as a Scalable Alternative to Reinforcement Learning*, 2017.